In [ ]:
import json
import os
from collections import defaultdict
from IPython.display import HTML
import numpy as np
import matplotlib
from matplotlib import animation
import matplotlib.pyplot as plt

from analysis import (
    parse_results, get_token_groups, plot_token_groups, animate, average_length_by_weight,
    tokens, counter_tokens, total_of, sum_up, mk_cumulative_interpolator
)
%matplotlib inline

In [ ]:
def read(path):
    tout, Cout, info = parse_results(path)
    tokg, ctrg, spg = get_token_groups(Cout)
    init = []
    for sk in spg['polymeric']:
        if Cout[sk][0] > 0:
            init.append((sk, Cout[sk][0]))
    (sk0, ic0), = init
    return locals()

In [ ]:
TOKEN = 'X3'
cases = __import__('cases_%s' % TOKEN)
def res(*, mf, atmo, tpulse):
    if tpulse == 0:
        return 'X1-res/X1_{mf}_{atmo}_1.txt'.format(mf=mf, atmo=atmo)
    else:
        return 'X3-res/X3_{mf}_{atmo}_{tpulse}.txt'.format(mf=mf, atmo=atmo, tpulse=tpulse-1)
#tps = range(len(cases.tpulses))
tps = [0, 1, 2]

In [ ]:
def _varied(tpulse_idx):
    if tpulse_idx == 0:
        return 'varied-X1-1.json'
    else:
        return 'varied-%s-%d.json' % (TOKEN, tpulse_idx-1)

interpols = []
for tpulse_idx in tps:
    dur, varied = json.load(open(_varied(tpulse_idx)))
    dr = varied.pop('doserate')
    assert not varied
    interpols.append(mk_cumulative_interpolator(dur, dr))

In [ ]:
all_air, all_nit = {}, {}
for mf in range(2):
    for atmo in range(2):
        for tpulse in tps:
            if atmo == 0:
                r = all_air
            elif atmo == 1:
                r = all_nit
            else:
                raise ValueError("Unexpected atmosphere index")
            r[mf, tpulse] = read(res(mf=mf, atmo=atmo, tpulse=tpulse))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 6))
xi = tps[2]
for i, k in enumerate(all_air[0, 0]['spg']['ord_rad']):
    ax.semilogy(all_air[0, xi]['tout'], all_air[0, xi]['Cout'][k], ls=['-',':','-.'][i % 3], label=k)
ax.set_ylim([1e-15, 100])
ax.legend()

In [ ]:
#plot_token_groups(tout, Cout, tokg)

In [ ]:
#anim1 = animate(tout, Cout, tokg)
#HTML(anim1.to_html5_video())

In [ ]:
def plot_conc(series_air, series_nit, ip1d, sks='H2O2 O2 N2O HO2'.split()):
    fig, all_axes = plt.subplots(len(series_air), 1 + len(sks), figsize=(16, 3*len(series_air)), sharey='col')
    assert len(series_air) == len(series_nit)
    for axes, air, nit in zip(all_axes, series_air, series_nit):
        for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
            axes[0].axhline(int(cont['sk0'][1:].split('a')[0]), ls='--', lw=0.5, alpha=0.5, c='k')
            axes[0].plot(ip1d(cont['tout'])/1e3, average_length_by_weight(cont['Cout'], cont['tokg']), label=lbl)
            for i in range(len(sks)):
                axes[i+1].plot(ip1d(cont['tout'])/1e3, cont['Cout'][sks[i]], label=lbl, alpha=0.7)
                axes[i+1].set_title(sks[i])
                axes[i+1].set_yscale('log')
        for ax in axes:
            ax.set_xlabel('Dose / kGy')
        axes[0].legend()
        axes[1].legend()
        _ = axes[0].set_title(r'Average length by weight')

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
if TOKEN == 'W2':
    for scal in range(5):
        plot_conc([all_air[scal, 0], all_air[scal, 1]], [all_nit[scal, 0], all_nit[scal, 1]])
elif TOKEN == 'W3':
    plot_conc(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])
elif TOKEN == 'X3':
    for tpulse in tps:
        cx, cy, ip1d = interpols[tpulse]
        plot_conc(*[[cont[mf, tpulse] for mf in range(len(cases.mass_fracs))] for cont in [all_air, all_nit]], ip1d)
        plt.suptitle("t_pulse = %.1e s" % (5e-6 if tpulse == 0 else cases.tpulses[tpulse-1]))
        plt.tight_layout()
        plt.subplots_adjust(left=0.1, wspace=0.9, top=0.9)

In [ ]:
def plot_tokens(series_air, series_nit, ip1d):
    assert len(series_air) == len(series_nit)
    fig, all_axes = plt.subplots(len(series_air), len(tokens[1:]), figsize=(16, 3*len(series_air)), sharey='col')
    for axes, air, nit in zip(all_axes, series_air, series_nit):
        for tk, ax in zip(tokens[1:], axes):
            for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
                ax.plot(ip1d(cont['tout']), total_of(cont['Cout'], cont['tokg'][tk])/cont['ic0'], label=lbl)
            ax.legend()
            ax.set_title(tk)
            ax.set_xlabel('Dose / Gy')
            ax.set_yscale('log')
    

In [ ]:
len(all_air[0, 0]['Cout']['H'])

In [ ]:
if TOKEN == 'W2':
    for scal in range(5):
        plot_tokens([all_air[scal, 0], all_air[scal, 1]], [all_nit[scal, 0], all_nit[scal, 1]])
elif TOKEN == 'W3':
    plot_tokens(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])
elif TOKEN == 'X3':
    for tpulse in tps:
        cx, cy, ip1d = interpols[tpulse]
        plot_tokens(*[[cont[mf, tpulse] for mf in range(len(cases.mass_fracs))] for cont in [all_air, all_nit]], ip1d)
        plt.suptitle("t_pulse = %.1e s" % (5e-6 if tpulse == 0 else cases.tpulses[tpulse-1]))
        plt.tight_layout()
        plt.subplots_adjust(left=0.1, wspace=0.9, top=0.9)    

In [ ]:
def plot_counters(series_air, series_nit, ip1d):
    assert len(series_air) == len(series_nit)
    fig, all_axes = plt.subplots(len(series_air), len(counter_tokens), figsize=(16, 3*len(series_air)), sharey='col')
    for axes, air, nit in zip(all_axes, series_air, series_nit):
        for ct, ax in zip(counter_tokens, axes):
            for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
                ax.plot(ip1d(cont['tout']), sum_up(cont['Cout'], cont['ctrg'][ct])/cont['ic0'], label=lbl)
            ax.set_xlabel('Dose / Gy')
            ax.legend()
            ax.set_title(ct)
            #ax.set_ylim([200, 300.]) ###############!!!!!!!!!!!!

In [ ]:
if TOKEN == 'W2':
    for scal in range(5):
        plot_counters([all_air[scal, 0], all_air[scal, 1]], [all_nit[scal, 0], all_nit[scal, 1]])
elif TOKEN == 'W3':
    plot_counters(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])
elif TOKEN == 'X3':
    for tpulse in tps:
        cx, cy, ip1d = interpols[tpulse]
        plot_counters(*[[cont[mf, tpulse] for mf in range(len(cases.mass_fracs))] for cont in [all_air, all_nit]], ip1d)
        plt.suptitle("t_pulse = %.1e s" % (5e-6 if tpulse == 0 else cases.tpulses[tpulse-1]))
        plt.tight_layout()
        plt.subplots_adjust(left=0.1, wspace=0.9, top=0.9)

In [ ]:
# This cell generates figures for Mats' conference presentation
parameter = dict(X1='freqs', X2='doserates', X3='tpulses')
offset = dict(X1=0, X2=1, X3=0)[TOKEN]
param_pretty = dict(freqs='Frequency', doserates='Doserate', tpulses='Pulse length')
param_unit = dict(freqs='Hz', doserates='Gy/s', tpulses='s')
vark = parameter[TOKEN]
varl = dict(X1=[], X2=[32e6], X3=[5e-6])[TOKEN] + getattr(cases, vark)
from chempy.printing import number_to_scientific_latex

for xi in range(len(varl)-offset):
    cx, cy, ip1d = interpols[xi]
    mfi = len(cases.mass_fracs)-1
    ppm = cases.mass_fracs[mfi]*1e6
    assert int(ppm) == 5000
    air, nit = all_air[mfi, xi], all_nit[mfi, xi]
    t_end_s = air['tout'][-1]
    print('t_end = %.5f s' % t_end_s)
    fig, ax = plt.subplots(1, 1)
    for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
        n0 = int(cont['sk0'][1:].split('a')[0])  # intial lenght
        ax.axhline(n0, ls='--', lw=0.5, alpha=0.5, c='k')
        ax.plot(ip1d(cont['tout'])/1e3, average_length_by_weight(cont['Cout'], cont['tokg']), label=lbl)
    ax.set_ylabel('Chain length / number of segments')
    ax.set_xlabel('Dose / kGy')
    ax.legend()
    ax.set_ylim([90, 800])
    
    varv = varl[xi]
    ax.set_title('{} = ${}$ {}'.format(param_pretty[vark], number_to_scientific_latex(varv), param_unit[vark]))
    fnam = lambda k: 'poly_simple_%s/{}_{}'.format(vark, varv) % k
    fig.savefig('%s.png' % fnam('size'), dpi=300)
    dur, varied = json.load(open(_varied(xi)))
    n_pulse = len(dur)/2
    dr = varied['doserate'][0]
    with open('%s.txt' % fnam('size'), 'wt') as fh:
        fh.write("t_end_s={}, freq_Hz={}, t_pulse_s={}, t_wait_s={}, doserate_Gy_s={:.4g}".format(
            t_end_s, n_pulse/t_end_s, dur[0], dur[1], dr
        ))
        
    # H2O2
    fig2, ax2 = plt.subplots(1, 1)
    for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
        ax2.plot(ip1d(cont['tout'])/1e3, cont['Cout']['H2O2'], label=lbl, alpha=0.7)
    ax2.set_ylabel(r'[$\mathrm{H_2O_2}$] / M')
    ax2.set_xlabel('Dose / kGy')
    ax2.set_ylim([0, 2e-3])
    ax2.legend()
    fig2.savefig('%s.png' % fnam('H2O2'), dpi=300)

    # unsaturated
    fig3, ax3 = plt.subplots(1, 1)
    for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
        ax3.plot(ip1d(cont['tout']), sum_up(cont['Cout'], cont['ctrg']['unsaturated'])/cont['ic0'], label=lbl)
    ax3.set_ylabel('Number of double bonds per polymer')
    ax3.set_xlabel('Dose / kGy')
    ax3.set_ylim([0, 400])
    ax3.legend()
    fig3.savefig('%s.png' % fnam('unsaturated'), dpi=300)